# Init

In [1]:
import sys
import os
import json
import time
import re

from tqdm import tqdm

cwd = os.getcwd()
os.chdir(cwd)
sys.path.append('tools')

import chat
import parse_data
import sql

In [26]:
api_key = "43d1f290abf2441ea6b52b6e1ef95e79"

import requests
import json

def fetch_data(data: dict):
    url = "https://comm.chatglm.cn/finglm2/api/query"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    response = requests.post(url, headers=headers, json=data)

    return response.json()

# Craft Prompt

In [9]:
system_prompt = ""

prompt_dir = os.path.join(cwd, 'prompt')
version = 'v2.0.0'
fname = f'sql_generator-stage_1-{version}.md'
prompt_fpath = os.path.join(prompt_dir, fname)

with open(prompt_fpath, 'r') as f:
    prompt_template = ''.join(f.readlines())

def make_prompt(data: dict) -> str:

    prompt = prompt_template

    # 
    table_finder_res = data['table_finder']['stage_1'][0]['data_source'][0]
    try:
        del table_finder_res['question']
    except:
        pass
    table = table_finder_res['table']
    table_finder_res = json.dumps(table_finder_res, ensure_ascii=False, indent=2)
    reg_p = re.compile('<Database and Table>')
    prompt = re.sub(reg_p, table_finder_res, prompt)

    # 
    table_fname = f'{table}-with_table_name.md'
    table_dir = os.path.join(cwd, 'data' + os.sep + 'table-column')
    table_fpath = os.path.join(table_dir, table_fname)
    with open(table_fpath,'r') as f:
        table_schema = ''.join(f.readlines())
    reg_p = re.compile('<Table-Column Schema>')
    prompt = re.sub(reg_p, table_schema, prompt)

    # 
    if data['ner']['stage_1']['result']:
        ner_res = [i for i in data['ner']['stage_1']['sql'].values() if i][0]
        ner_res = [i['result'] for i in ner_res if i['result']][0][0]
        ner_res = json.dumps(ner_res, ensure_ascii=False, indent=2)
        reg_p = re.compile('<Background Knowledge>')
        prompt = re.sub(reg_p, ner_res, prompt)
    else:
        reg_p = re.compile('<Background Knowledge>')
        prompt = re.sub(reg_p, '', prompt)
        reg_p = re.compile('## Background Knowledge')
        prompt = re.sub(reg_p, '', prompt)

    # replace query
    query = data['team'][0]['question']
    reg_p = re.compile('<Current Query>')
    prompt = re.sub(reg_p, query, prompt)

    return prompt

In [3]:
question_path = os.path.join(cwd, 'answer_tmp' + os.sep + 'stage_1-glm_4_plus-table_finder-v2.7.1.json')

questions = parse_data.read_json(question_path)

# GLM-4-Plus

In [4]:
model = 'glm_4_plus'

## Test

In [ ]:
query = make_prompt(questions[1])

history = []

start_time = time.time()
message = chat.create_message(query, history=history, system_prompt=system_prompt, temperature=0.7, top_p=0.9, response_format='text')
end_time = time.time()

execution_time = end_time - start_time
usage = chat.get_token_usage(message, True)
content = chat.get_content(message, True)
history = chat.build_history(history, message=message)

In [6]:
t = questions[1]
t['sql_generator'] = {}
t['sql_generator']['stage_1']= [json.loads(content.strip('`json'))]
t['token_usage'] = {}
t['token_usage']['sql_generator-stage_1'] = [usage]
t['time_usage'] = {}
t['time_usage']['sql_generator-stage_1'] = [f"{execution_time:.2f}s"]
t = [t]

saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + f'stage_1-{model}-sql_generator-test-{version}.json')
parse_data.write_json(t, saved_path)

## ALL

In [ ]:
answers = []

for question in tqdm(questions[:]):
    try:
        # the first question
        query = make_prompt(question)

        history = []

        start_time = time.time()
        message = chat.create_message(query, history=history, system_prompt=system_prompt, temperature=0.7, top_p=0.9, response_format='text')
        end_time = time.time()

        execution_time = end_time - start_time
        usage = chat.get_token_usage(message, False)
        content = chat.get_content(message, False)

        res = question
        res['sql_generator'] = {}
        res['sql_generator']['stage_1']= [json.loads(content.strip('`json'))]
        res['token_usage']['sql_generator-stage_1'] = [usage]
        res['time_usage']['sql_generator-stage_1'] = [f"{execution_time:.2f}s"]

        answers.append(res)
    except:
        print(question['tid'])

saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + f'stage_1-{model}-sql_generator-{version}.json')
parse_data.write_json(answers, saved_path)

# Deepseek-chat

In [12]:
model = 'deepseek_v3'

In [13]:
deepseek_api = 'sk-ba0f5eed3bea4fa6be16eb33b139c684'

## Test

In [14]:
from openai import OpenAI

query = make_prompt(questions[1])

client = OpenAI(api_key= deepseek_api, base_url="https://api.deepseek.com")

start_time = time.time()
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "user", "content": query},
    ],
    stream=False,
    top_p=0.7,
    temperature=0.9
)
end_time = time.time()

response = json.loads(response.to_json())
content = response['choices'][0]['message']['content']

content = content.strip('`json')
usage = response['usage']
execution_time = end_time - start_time

In [15]:
t = questions[1]
t['sql_generator'] = {}
t['sql_generator']['stage_1']= [json.loads(content.strip('`json'))]
t['token_usage'] = {}
t['token_usage']['sql_generator-stage_1'] = [usage]
t['time_usage'] = {}
t['time_usage']['sql_generator-stage_1'] = [f"{execution_time:.2f}s"]
t = [t]

saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + f'stage_1-{model}-sql_generator-test-{version}.json')
parse_data.write_json(t, saved_path)

# ALL

In [20]:
answers = []

for question in tqdm(questions[:]):

    max_retries = 5
    retries = 0

    res = question
    res['sql_generator'] = {}

    while retries < max_retries:
        try:
            query = make_prompt(question)

            start_time = time.time()
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=[
                    {"role": "user", "content": query},
                ],
                stream=False,
                top_p=0.7,
                temperature=0.9
            )
            end_time = time.time()

            response = json.loads(response.to_json())
            content = response['choices'][0]['message']['content']

            content = content.strip('`json')
            usage = response['usage']
            execution_time = end_time - start_time

            res['sql_generator']['stage_1']= [json.loads(content.strip('`json'))]
            res['token_usage']['sql_generator-stage_1'] = [usage]
            res['time_usage']['sql_generator-stage_1'] = [f"{execution_time:.2f}s"]

            data = {
                "sql": json.loads(content.strip('`json'))['sql_query'],
                "limit": 100
            }
            sql_res = fetch_data(data)['data']
            
            if sql_res:
                answers.append(res)
                break

        except Exception as e:
            retries += 1
            print(f"Error processing question {question['tid']} (attempt {retries}/{max_retries}): {e}")
            if retries == max_retries:
                print(f"Failed to process question {question['tid']} after {max_retries} attempts.")
                answers.append(res)
        
saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + f'stage_1-{model}-sql_generator-{version}.json')
parse_data.write_json(answers, saved_path)

 14%|█▍        | 14/101 [05:57<17:20, 11.96s/it] 

Error processing question tttt----9 (attempt 1/5): 'data'
Error processing question tttt----9 (attempt 2/5): 'data'
Error processing question tttt----9 (attempt 3/5): 'data'
Error processing question tttt----9 (attempt 4/5): 'data'


 15%|█▍        | 15/101 [07:01<39:43, 27.72s/it]

Error processing question tttt----9 (attempt 5/5): 'data'
Failed to process question tttt----9 after 5 attempts.


 16%|█▌        | 16/101 [07:09<30:43, 21.69s/it]

...

 95%|█████████▌| 96/101 [34:53<02:31, 30.31s/it]

Error processing question tttt----91 (attempt 1/5): 'data'


 97%|█████████▋| 98/101 [35:40<01:17, 25.90s/it]

Error processing question tttt----98 (attempt 1/5): 'data'
Error processing question tttt----98 (attempt 2/5): 'data'
Error processing question tttt----98 (attempt 3/5): 'data'
Error processing question tttt----98 (attempt 4/5): 'data'


100%|██████████| 101/101 [37:02<00:00, 22.00s/it]


# Obtain SQL Results

## GLM-4-Plus

In [17]:
model = 'glm_4_plus'

In [ ]:
fname = f'stage_1-{model}-sql_generator-{version}.json'
fpath = os.path.join(cwd, 'answer_tmp' + os.sep + fname)
data = parse_data.read_json(fpath)

for i in tqdm(data[:]):
    tmp = sql.process_sql_generator_res(i['sql_generator']['stage_1'][0])

fname = f'stage_1-{model}-sql_generator-{version}-sql.json'
fpath = os.path.join(cwd, 'answer_tmp' + os.sep + fname)
data = parse_data.write_json(data, fpath)

## Deepseek-v3

In [21]:
model = 'deepseek_v3'

In [28]:
fname = f'stage_1-{model}-sql_generator-{version}.json'
fpath = os.path.join(cwd, 'answer_tmp' + os.sep + fname)
data = parse_data.read_json(fpath)

for i in tqdm(data[:]):
    try:
        tmp = sql.process_sql_generator_res(i['sql_generator']['stage_1'][0])
    except Exception as e:
        print(e)

fname = f'stage_1-{model}-sql_generator-{version}-sql.json'
fpath = os.path.join(cwd, 'answer_tmp' + os.sep + fname)
data = parse_data.write_json(data, fpath)

  0%|          | 0/101 [00:00<?, ?it/s]

 16%|█▌        | 16/101 [00:03<00:19,  4.37it/s]

Request failed: 500 Server Error: Internal Server Error for url: https://comm.chatglm.cn/finglm2/api/query


 17%|█▋        | 17/101 [00:03<00:19,  4.38it/s]

Request failed: 500 Server Error: Internal Server Error for url: https://comm.chatglm.cn/finglm2/api/query


...

 64%|██████▍   | 65/101 [00:16<00:10,  3.42it/s]

'stage_1'


 91%|█████████ | 92/101 [00:24<00:02,  4.10it/s]

'stage_1'


100%|██████████| 101/101 [00:27<00:00,  3.63it/s]
